In [1]:
import numpy as np
import pandas as pd

from fastparquet import ParquetFile,write
import matplotlib.pyplot as plt
from matplotlib import cm
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from cm2df import cm2df,precision_recall_fscore_support_metrics2df
from sklearn.metrics import confusion_matrix, classification_report,precision_recall_fscore_support
import pickle
from bson.json_util import dumps
from tqdm import tqdm
import re

In [96]:
filename = 'finalized_model111819.sav'
# loading the model from disk
model = pickle.load(open(filename, 'rb'))

In [3]:
usecolsX=['category', 'past', 'is_eventbrite', 'is_free', 'doors',
       'sold_out', 'venue.id', 'venue.popularity', 'venue.zip',
       'ticket_allages', 'ticket_price_low', 'ticket_price_max', 'min_age',
       'artist.popularity.sum', 'artist.popularity.avg',
       'artist.popularity.max', 'dow', 'doy', 'month', 'day', 'hour',
       'venue.tol_num_events', 'duration', 'duration_day', 'multiday']
usecoly1=['votes']
usecoly2=['avg_votes_pday']

In [4]:
import pymongo
import json
import pprint
from pymongo import MongoClient, GEO2D
import datetime
import json
import pprint

### read from daily do512 json file since collected database does not contain some info for features

In [181]:
# start off the dataframe 
ncolumns=['id','event_name','category','past', 'is_eventbrite', 'is_free', 'doors','sold_out', 'venue.id', 'venue.popularity', 'venue.zip','ticket_info', 'artist','tz_adjusted_begin_date','tz_adjusted_end_date','votes']
ev_df=pd.DataFrame(columns=ncolumns)

In [163]:
# %load NM_get_do512_future_data.py
# Get the event data from the do 512 website for the future N days
import time
from datetime import datetime, timedelta
import json
import requests
from pandas.io.json import json_normalize

def get_request_url(days_to_add,page):
    base_url = 'https://do512.com/events/'
    date = datetime.now() + timedelta(days=days_to_add)
    print(date)
    url_suffix = str(date.year) + '/' + str(date.month) + '/' + str(date.day) + '.json'
    page_num = '?page=' + str(page)
    request_url = base_url + url_suffix + page_num
    return request_url


def get_events(days_to_add,page):
    request_url = get_request_url(days_to_add,page)
    response = requests.get(request_url)
    return response

def process_response(response):
    events = json.loads(response)["events"]
    return events

def get_total_pages(response):
    total_pages = json.loads(response)["paging"]["total_pages"]
    return total_pages

def date_to_str(date):
    return str(date.month) + '-' + str(date.day) + '-' + str(date.year)

def get_file_name(NumDaysInFut):
    folder_name = 'C:\SiemensCodeSpace\Eventful\PredictionModels\data\\'
    base_name = 'do512_'
    name_suffix = date_to_str(datetime.now()) + "-"  + date_to_str(datetime.now() + timedelta(days=(NumDaysInFut-1)))
    file_name = folder_name + base_name + name_suffix + '.parquet'
    return file_name

In [182]:
query= get_events(-22,1)

2019-10-29 15:28:58.253374


In [183]:
pages= query.json()['paging']['total_pages']

In [184]:
for p in range(pages):
    query= get_events(-22,p+1)
    content=query.json()
    events=content['events']
    print("number of events on the page:",len(events))
    for evt in events:
        # new list
        ev_list=[]
        ev_list+=[evt['id']]
        ev_list+=[evt['title']]
        ev_list+=[evt['category']]
        ev_list+=[evt['past']]
        ev_list+=[evt['is_eventbrite']]
        ev_list+=[evt['is_free']]
        ev_list+=[evt['doors']]
        ev_list+=[evt['sold_out']]
        ev_list+=[evt['venue']['id']]
        ev_list+=[evt['venue']['popularity']]
        ev_list+=[evt['venue']['zip']]
        ev_list+=[evt['ticket_info']]
        ev_list+=[evt['artists']]
        ev_list+=[evt['tz_adjusted_begin_date']]
        ev_list+=[evt['tz_adjusted_end_date']]
        ev_list+=[evt['votes']]
        #save list to dataframe
        ev_df.loc[len(ev_df.index)] = ev_list    

2019-10-29 15:29:00.918407
number of events on the page: 25
2019-10-29 15:29:01.275227
number of events on the page: 25
2019-10-29 15:29:02.291954
number of events on the page: 21


In [186]:
print("total number of events:",len(ev_df))

total number of events: 71


In [187]:
ev_df.head()

,id,event_name,category,past,is_eventbrite,is_free,doors,sold_out,venue.id,venue.popularity,venue.zip,ticket_info,artist,tz_adjusted_begin_date,tz_adjusted_end_date,votes
0,11129312,The Film Pass Party,Film + TV,True,False,False,False,False,114333,1.0,78701,$65 Film Pass,[],2019-10-29T23:00:00-05:00,2019-10-30T02:00:00-05:00,668
1,11199198,THE ROCKY HORROR PICTURE SHOW,Film + TV,True,False,False,False,False,268172,1.0,,,[],2019-10-29T20:00:00-05:00,2019-10-30T02:00:00-05:00,128
2,11124772,"Ski Mask the Slump God w/ Pouya, Pop Smoke & More",Music,True,False,False,True,False,175439,15.0,78741,All Ages,"[{'id': 996081, 'title': 'Ski Mask the Slump G...",2019-10-29T19:00:00-05:00,2019-10-30T02:00:00-05:00,72
3,11046923,Screaming Females + Dusk with The Naked Tungs,Music,True,False,False,False,False,19704,15.0,78701,$18-$20,"[{'id': 48016, 'title': 'Screaming Females', '...",2019-10-29T21:00:00-05:00,2019-10-30T02:00:00-05:00,72
4,11190075,Mike Stinson,Music,True,False,False,False,False,314,10.0,78704,"$7, Ages 21+","[{'id': 37781, 'title': 'Mike Stinson', 'perma...",2019-10-29T22:00:00-05:00,2019-10-30T02:00:00-05:00,8


In [188]:
vzip=ev_df['venue.zip'].values
# cleanup the zipcodes
vzip=[str(x).split(' ')[0] for x in vzip]
vzip=['' if x=='Austin' else x for x in vzip]
vzip=['' if x=='nan' else x for x in vzip]
vzip=['' if len(x)<5 else x for x in vzip]

In [189]:
vzip=[int(x) if x.isdigit() else 0 for x in vzip]

In [190]:
ev_df['venue.zip']=vzip

In [191]:
# clean up ticket infomation
# insert new columns
ev_df['ticket_allages']=[False for x in range(len(ev_df))]
ev_df['ticket_price_low']=[0 for x in range(len(ev_df))]
ev_df['ticket_price_max']=[0 for x in range(len(ev_df))]
ev_df['min_age']=[None for x in range(len(ev_df))]

# parse ticket info string
tkt_str=ev_df.ticket_info.values

# dump all info of ticket into other columns
for i,item in tqdm(enumerate(tkt_str)):
    if i%1000==0:print(i)
    #check 'free'
    if "free" in item.lower() : ev_df.loc[i,'is_free']=True
    # check child ok        
    if ("all ages" in item.lower()) or ("kids" in item.lower()) or ("child" in item.lower()): ev_df.loc[i,'ticket_allages']=True
    
    #replace all whitespace after '$'    
    item = re.sub('\$ ','$',item)
    prices=re.findall('(?<=\$)\d*\.?\d*', item)
    # get all lower/upper $number
    if prices: # not [] or None
        if prices[0].isdigit():
            if len(prices)==1: 
                ev_df.loc[i,'ticket_price_low']=prices[0]
                ev_df.loc[i,'ticket_price_max']=None
            else:
                #convert all prices to numbers , in case any of them not number
                nprices=[float(x) for x in prices if x.isdigit()]
                
                ev_df.loc[i,'ticket_price_low']=min(nprices)
                ev_df.loc[i,'ticket_price_max']=max(nprices)      
    # get minimum age
    min_ag=re.findall(r"(\d+)\+",item)
    if min_ag:
        ev_df.loc[i,'min_age']=int(min_ag[0])
    else:ev_df.loc[i,'min_age']=None

0it [00:00, ?it/s]

0


71it [00:00, 1617.94it/s]


In [192]:
# parse artist information
# get artist column andef
e_artist=ev_df.artist.values
#get artist_popularity
p_art_avg=[]
p_art_sum=[]
p_art_max=[]
for item in e_artist:
    if len(item)==0:
        p_art_sum+=[0]
        p_art_avg+=[0]
        p_art_max+=[0]
    elif len(item)==1:
        p_art_sum+=[json.loads(dumps(item))[0]['popularity']]
        p_art_avg+=[json.loads(dumps(item))[0]['popularity']]
        p_art_max+=[json.loads(dumps(item))[0]['popularity']]
    else:
        subsum=0
        partmax=0
        for sub in item:
            part=json.loads(dumps(item))[0]['popularity']
            subsum+=part
            if part> partmax: partmax=part
        p_art_sum+=[subsum]
        p_art_avg+=[subsum/len(item)]
        p_art_max+=[partmax]

In [193]:
ev_df['artist.popularity.sum']=p_art_sum
ev_df['artist.popularity.avg']=p_art_avg
ev_df['artist.popularity.max']=p_art_max

In [194]:
# setting the time and get the time features
ev_df['tz_adjusted_begin_date']=pd.to_datetime(ev_df['tz_adjusted_begin_date'])
ev_df['tz_adjusted_end_date']=pd.to_datetime(ev_df['tz_adjusted_end_date'])


ev_df['duration']=(ev_df['tz_adjusted_end_date']-ev_df['tz_adjusted_begin_date']).apply(lambda x:x/ np.timedelta64(1, 'h'))
ev_df['duration_day']=ev_df['duration'].apply(lambda x:np.ceil(x/24))
a=ev_df['duration'].values/24
multid=[True if x >=1 else False for x in a]
ev_df['multiday']=multid
# fix duration_day=0
ev_df=ev_df.replace({'duration':0},2)
ev_df=ev_df.replace({'duration_day':0},1)

In [195]:
ev_df['dow']=ev_df['tz_adjusted_begin_date'].dt.dayofweek
ev_df['doy']=ev_df['tz_adjusted_begin_date'].dt.dayofyear
ev_df['month']=ev_df['tz_adjusted_begin_date'].dt.month
ev_df['day']=ev_df['tz_adjusted_begin_date'].dt.day
ev_df['hour']=ev_df['tz_adjusted_begin_date'].dt.hour

In [196]:
### drop some redundant columns
ev_df.drop(['ticket_info', 'artist','tz_adjusted_begin_date','tz_adjusted_end_date'], axis=1, inplace=True)

In [197]:
ev_df.head()

,id,event_name,category,past,is_eventbrite,is_free,doors,sold_out,venue.id,venue.popularity,...,artist.popularity.avg,artist.popularity.max,duration,duration_day,multiday,dow,doy,month,day,hour
0,11129312,The Film Pass Party,Film + TV,True,False,False,False,False,114333,1.0,...,0.0,0.0,3.0,1.0,False,1,302,10,29,23
1,11199198,THE ROCKY HORROR PICTURE SHOW,Film + TV,True,False,False,False,False,268172,1.0,...,0.0,0.0,6.0,1.0,False,1,302,10,29,20
2,11124772,"Ski Mask the Slump God w/ Pouya, Pop Smoke & More",Music,True,False,False,True,False,175439,15.0,...,59.0,59.0,7.0,1.0,False,1,302,10,29,19
3,11046923,Screaming Females + Dusk with The Naked Tungs,Music,True,False,False,False,False,19704,15.0,...,768.0,768.0,5.0,1.0,False,1,302,10,29,21
4,11190075,Mike Stinson,Music,True,False,False,False,False,314,10.0,...,15.0,15.0,4.0,1.0,False,1,302,10,29,22


In [198]:
### Add the total events host at the venue , query from past year library
# load back training file
toevv=pd.read_csv('Dataset_Lib/TotalEventsPerVenueID2018.csv')

In [199]:
toevv=toevv.set_index('id')

In [200]:
# query venue id then input number
tolnev=[]
venid=ev_df['venue.id'].values
lvenid=toevv['venue.id'].values
for idn in venid:
    if idn in lvenid:tolnev+=[toevv.loc[toevv['venue.id']==idn].iloc[0,0]]
    else:tolnev+=[1]

In [201]:
ev_df['venue.tol_num_events']=tolnev

In [202]:
ev_df.past=ev_df.past.astype(bool)
ev_df.is_eventbrite=ev_df.is_eventbrite.astype(bool)
ev_df.doors=ev_df.doors.astype(bool)
ev_df.sold_out=ev_df.sold_out.astype(bool)

In [203]:
ev_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71 entries, 0 to 70
Data columns (total 28 columns):
id                       71 non-null object
event_name               71 non-null object
category                 71 non-null object
past                     71 non-null bool
is_eventbrite            71 non-null bool
is_free                  71 non-null bool
doors                    71 non-null bool
sold_out                 71 non-null bool
venue.id                 71 non-null object
venue.popularity         71 non-null float64
venue.zip                71 non-null int64
votes                    71 non-null object
ticket_allages           71 non-null bool
ticket_price_low         71 non-null object
ticket_price_max         50 non-null float64
min_age                  15 non-null object
artist.popularity.sum    71 non-null float64
artist.popularity.avg    71 non-null float64
artist.popularity.max    71 non-null float64
duration                 71 non-null float64
duration_day            

In [204]:
ev_df.head()

,id,event_name,category,past,is_eventbrite,is_free,doors,sold_out,venue.id,venue.popularity,...,artist.popularity.max,duration,duration_day,multiday,dow,doy,month,day,hour,venue.tol_num_events
0,11129312,The Film Pass Party,Film + TV,True,False,False,False,False,114333,1.0,...,0.0,3.0,1.0,False,1,302,10,29,23,62
1,11199198,THE ROCKY HORROR PICTURE SHOW,Film + TV,True,False,False,False,False,268172,1.0,...,0.0,6.0,1.0,False,1,302,10,29,20,1
2,11124772,"Ski Mask the Slump God w/ Pouya, Pop Smoke & More",Music,True,False,False,True,False,175439,15.0,...,59.0,7.0,1.0,False,1,302,10,29,19,116
3,11046923,Screaming Females + Dusk with The Naked Tungs,Music,True,False,False,False,False,19704,15.0,...,768.0,5.0,1.0,False,1,302,10,29,21,481
4,11190075,Mike Stinson,Music,True,False,False,False,False,314,10.0,...,15.0,4.0,1.0,False,1,302,10,29,22,251


In [205]:
# rearrange column orders:
ev_df=ev_df[usecolsX+['votes']]

In [206]:
ev_df.to_csv('Dataset_Lib/ev_do512_102919.csv')

#### Null Data cleanup

In [209]:
ev_df.isnull().sum()

category                 0
past                     0
is_eventbrite            0
is_free                  0
doors                    0
sold_out                 0
venue.id                 0
venue.popularity         0
venue.zip                0
ticket_allages           0
ticket_price_low         0
ticket_price_max         0
min_age                  0
artist.popularity.sum    0
artist.popularity.avg    0
artist.popularity.max    0
dow                      0
doy                      0
month                    0
day                      0
hour                     0
venue.tol_num_events     0
duration                 0
duration_day             0
multiday                 0
votes                    0
dtype: int64

In [208]:
ev_df.min_age=ev_df.min_age.fillna(0)
ev_df.ticket_price_max=ev_df.ticket_price_max.fillna(0)
ev_df.ticket_price_low=ev_df.ticket_price_low.fillna(0)

#### Ecoding data for input:
(1) Convert boolean columns
(2) Convert/map category,
(3) Map the venue id to nullify
(4) Map the zipcode to nullify

In [210]:
boolcol = ev_df.columns[(ev_df.dtypes.values == np.dtype('bool'))]
## convert all boolean columns into 0/1
for cols in boolcol:
    ev_df[cols]=ev_df[cols].astype(int)

In [211]:
### Conversion of the features
# mapping of the category
mapcat={'Music ': 0, 'Comedy': 1, 'Happy Hour + Drink Specials': 2, "DJ's + Parties": 3, 'Film + TV': 4, 'Karaoke + Trivia': 5, 'Workshops + Classes': 6, 'Community': 7, 'Literary': 8, 'Art & Culture': 9, 'Sports + Activities': 10, 'Food + Drink': 11, 'LGBTQ+': 12, 'Theater': 13, 'Variety / Other': 14, 'Activism': 15, 'Exhibit': 16, 'Opening': 17, 'Fashion': 18, 'Free Week': 19}

for i,line in enumerate(mapcat):
    ev_df.category.replace(line,i,inplace=True)

In [215]:
# load back the venue id library
encode_dict=np.load('Dataset_Lib/venue_id_lib.npy',allow_pickle=True).item()

In [216]:
ev_df['venue.id']=ev_df['venue.id'].map(encode_dict)

In [217]:
nct=max(encode_dict.values())
fullvn_id=ev_df['venue.id'].values
nvn_id=[]
for item in fullvn_id:
    if np.isnan(item):
        nvn_id+=[int(nct+1)]
        nct+=1
    else: nvn_id+=[int(item)]

In [218]:
ev_df['venue.id']=nvn_id  # place back the venue id nullified

In [219]:
## load back the zip code library # neutrolize the venue zips:
encode_zip_dict=np.load('Dataset_Lib/venue_zip_lib.npy',allow_pickle=True).item()
ev_df['venue.zip']=ev_df['venue.zip'].map(encode_zip_dict)

In [220]:
# clean up the nan
nct=max(encode_zip_dict.values())
fullvn_zip=ev_df['venue.zip'].values
nvn_zip=[]
for item in fullvn_zip:
    if np.isnan(item):
        nvn_zip+=[int(nct+1)]
        nct+=1
    else: nvn_zip+=[int(item)]

In [221]:
ev_df['venue.zip']=nvn_zip

In [222]:
ev_df

,category,past,is_eventbrite,is_free,doors,sold_out,venue.id,venue.popularity,venue.zip,ticket_allages,...,dow,doy,month,day,hour,venue.tol_num_events,duration,duration_day,multiday,votes
0,4,1,0,0,0,0,121,1.0,0,0,...,1,302,10,29,23,62,3.000000,1.0,0,668
1,4,1,0,0,0,0,1614,1.0,3,0,...,1,302,10,29,20,1,6.000000,1.0,0,128
2,0,1,0,0,1,0,70,15.0,8,1,...,1,302,10,29,19,116,7.000000,1.0,0,72
3,0,1,0,0,0,0,5,15.0,0,0,...,1,302,10,29,21,481,5.000000,1.0,0,72
4,0,1,0,0,0,0,26,10.0,1,0,...,1,302,10,29,22,251,4.000000,1.0,0,8
5,3,1,0,0,1,0,11,15.0,0,0,...,1,302,10,29,20,405,6.000000,1.0,0,13
6,0,1,1,0,1,0,13,4.0,0,1,...,1,302,10,29,20,387,6.000000,1.0,0,57
7,1,1,0,0,1,0,49,15.0,4,0,...,1,302,10,29,20,167,2.500000,1.0,0,26
8,0,1,0,0,1,0,15,15.0,0,1,...,1,302,10,29,20,377,6.000000,1.0,0,33
9,0,1,0,1,0,0,48,15.0,2,0,...,1,302,10,29,21,173,5.000000,1.0,0,11


In [223]:
# Load back to the X scalar
import joblib
scalar = joblib.load('MinMaxScalar_111819.pkl')

In [224]:
udf=ev_df[usecolsX]

In [225]:
X=scalar.transform(udf)

In [226]:
# a function to create udf_y based on 3 classes: low, mid, high
def get_class(df,thre1,thre2):
    n=len(df)
    newdf=[]
    cut1=min([thre1,thre2])
    cut2=max([thre1,thre2])
    for item in df:
        if item<cut1: newdf+=[0]
        elif (item<cut2) and (item>=cut1):newdf+=[1]
        else: newdf+=[2]
    return newdf

In [227]:
y=get_class(ev_df['votes'].values,80,300)

In [228]:
y_pred=model.predict(X)

In [229]:
y_pred_prob=model.predict_proba(X)

In [230]:
result=list(zip(y,y_pred,ev_df['votes'].values,y_pred_prob[:,0],y_pred_prob[:,1],y_pred_prob[:,2]))

In [231]:
result_df=pd.DataFrame(result,columns=['class actual','class predicted','actual votes','probability in class0','probability in class1','probability in class2'])

In [232]:
result_df.sort_values(by=['actual votes'], ascending=False)

,class actual,class predicted,actual votes,probability in class0,probability in class1,probability in class2
0,2,0,668,0.922878,0.076839,0.000283
1,1,0,128,0.971920,0.001759,0.026321
2,0,0,72,0.487579,0.460615,0.051806
3,0,0,72,0.843075,0.145813,0.011111
6,0,0,57,0.518750,0.415218,0.066032
10,0,0,47,0.897123,0.100794,0.002083
8,0,0,33,0.906647,0.090496,0.002857
7,0,0,26,0.714256,0.275377,0.010367
11,0,0,15,0.844931,0.094980,0.060088
14,0,0,15,0.999638,0.000246,0.000116


In [233]:
result_df.sort_values(by=['class predicted','probability in class0'], ascending=False)

,class actual,class predicted,actual votes,probability in class0,probability in class1,probability in class2
62,0,0,1,0.999955,0.000043,0.000002
41,0,0,3,0.999924,0.000068,0.000008
53,0,0,1,0.999898,0.000102,0.000000
22,0,0,1,0.999856,0.000121,0.000023
31,0,0,4,0.999833,0.000123,0.000045
59,0,0,0,0.999803,0.000188,0.000009
57,0,0,1,0.999783,0.000131,0.000086
43,0,0,5,0.999778,0.000189,0.000032
45,0,0,2,0.999771,0.000220,0.000009
61,0,0,1,0.999722,0.000261,0.000017


In [234]:
#  check the confusion matrix
a=precision_recall_fscore_support(y, y_pred)
print("confusion matrix:")
print(confusion_matrix(y,y_pred))
print(classification_report(y,y_pred))

C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


confusion matrix:
[[69  0  0]
 [ 1  0  0]
 [ 1  0  0]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        69
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1

    accuracy                           0.97        71
   macro avg       0.32      0.33      0.33        71
weighted avg       0.94      0.97      0.96        71



In [235]:
f1s=a[2]
print("weighted average f1 score: ",(f1s[0]+f1s[1]*2+f1s[2]*3)/6)

weighted average f1 score:  0.16428571428571428


In [236]:
result_df.to_csv('102919_evdo512Pred_result.csv')